In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from CostGeneration import CostGeneration
from DataGeneration import DataGeneration

DataGeneration loads the data from:

https://data.open-power-system-data.org/time_series/ 

From this website the years 2015-2020 (2020 only includes data points from 1st of January - 30th of September) are downloaded.
For each hour in a given year the load demand, solar profile, wind profile on and offshore are downloaded for Denmark (DK1 and DK2), Germany and Norway

# Der er ikke noget data for wind_profile og solar_profile for DK og NO, så jeg har taget det fra capacity, og divideret med maks kapaciteten

In [15]:
import pypsa
import pandas as pd
from CostGeneration import CostGeneration
from DataGeneration import DataGeneration

class BuildBaseNetwork:
    def __init__(self, year: int = 2019, cost_year: int = 2030,
                 technology_data = dict, interest_rate = 0.07, 
                #  carriers: list[str] = ["onwind", "solar", "OCGT", "CCGT", "battery storage"], 
                #  multiple_regions = False,
                 setup: dict = {'DK': 
                            {'OCGT': True,
                            'CCGT': True,
                            'battery storage': True,
                            'onwind': True,
                            'solar': True}}):
        
        self.year = year

        self.cost_year = cost_year 
        self.costs = CostGeneration(year = self.cost_year).costs

        self.interest_rate = interest_rate

        self.setup = setup
        self.regions = setup.keys()
        
        self.network = pypsa.Network()
        self.hours_in_year = pd.date_range(f'{year}-01-01 00:00', f'{year}-12-31 23:00', freq='h')
        self.network.set_snapshots(self.hours_in_year.values)

        self.data_dict = {region : {} for region in self.regions}

        self.carriers = ['gas', 'onwind', 'offwind', 'solar']
        self.network.add("Carrier", self.carriers, color=["dodgerblue", "gold", "indianred", "yellow-green"], co2_emissions=[self.costs.at[c, "CO2 intensity"] for c in self.carriers])

        self.add_regions()

    def add_regions(self):
        for region in self.regions:
            data = DataGeneration(year = self.year, region = region)
            self.data_dict[region]['Demand'] = data.demand
            self.data_dict[region]['solar'] = data.solar
            self.data_dict[region]['onwind'] = data.onshore_wind
            self.data_dict[region]['offwind'] = data.offshore_wind 

            self.add_busses(region) # add bus to region bus

            technologies = self.setup[region].keys()
            for tech in technologies:
                self.add_network_technologies(region, tech) # add all regioin technologies

    def add_busses(self, region):
        self.network.add("Bus", f'electricity bus {region}')
        self.network.add("Load", f'load {region}', bus = f'electricity bus {region}', p_set = self.data_dict[region]['Demand'].values.flatten())

    def add_network_technologies(self, region, tech):
        if tech in ['OCGT', 'CCGT']:
            self.network.add("Generator", f'{tech} {region}', 
                                bus = f'electricity bus {region}', 
                                p_nom_extendable=True, 
                                carrier='gas', 
                                capital_cost = self.costs.at[tech, "capital_cost"], 
                                marginal_cost = self.costs.at[tech, "marginal_cost"])
        elif tech == 'solar':
            self.network.add("Generator",
                                f'{tech} {region}', 
                                bus = f'electricity bus {region}', 
                                p_nom_extendable=True, 
                                carrier='solar', 
                                capital_cost = self.costs.at[tech, "capital_cost"], 
                                marginal_cost = self.costs.at[tech, "marginal_cost"],
                                p_max_pu = self.data_dict[region]['solar'].values.flatten())     
        elif tech == 'onwind':
            self.network.add("Generator", f'{tech} {region}', 
                                bus = f'electricity bus {region}', 
                                p_nom_extendable=True, 
                                carrier='onwind', 
                                capital_cost = self.costs.at[tech, "capital_cost"], 
                                marginal_cost = self.costs.at[tech, "marginal_cost"],
                                p_max_pu = self.data_dict[region]['onwind'].values.flatten())  
        elif tech == 'offwind':
            self.network.add("Generator", f'{tech} {region}', 
                                bus = f'electricity bus {region}', 
                                p_nom_extendable=True, 
                                carrier='offwind', 
                                capital_cost = self.costs.at[tech, "capital_cost"], 
                                marginal_cost = self.costs.at[tech, "marginal_cost"],
                                p_max_pu = self.data_dict[region]['offwind'].values.flatten())

        
    # def add_line(self, name, bus0, bus1, length, s_nom):
    #     self.network.add("Line", name, bus0, bus1, length=length, s_nom=s_nom)

    @staticmethod
    def annuity(n, r):
        """ Calculate the annuity factor for an asset with lifetime n years and
        discount rate  r """

        if r > 0:
            return r/(1. - 1./(1.+r)**n)
        else:
            return 1/n


In [16]:
# example with DK
setup = {'DK': 
                {'OCGT': True,
                'solar': True,
                'offwind': True},
                }

tmp = BuildBaseNetwork(setup = setup)
tmp.network.optimize(solver_name="gurobi",solver_options={"OutputFlag": 0})

Year 2019 is greater than 2017, using data from last available year.


Index(['electricity bus DK'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.model:Solver options:
 - OutputFlag: 0
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 2/2 [00:00<00:00, 19.70it/s]
INFO:linopy.io: Writing time: 0.73s


Set parameter Username


INFO:gurobipy:Set parameter Username


Set parameter LicenseID to value 2638559


...

Reading time = 0.21 seconds


INFO:gurobipy:Reading time = 0.21 seconds


obj: 61323 rows, 26283 columns, 100760 nonzeros


INFO:gurobipy:obj: 61323 rows, 26283 columns, 100760 nonzeros
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 26283 primals, 61323 duals
Objective: 2.13e+09
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper were not assigned to the network.


('ok', 'optimal')

In [18]:
#tmp.network.loads_t.p_set
#tmp.network.generators_t.p_max_pu
print(tmp.network.generators.p_nom_opt)

Generator
OCGT DK       5383.879060
solar DK      1817.474339
offwind DK    4673.395990
Name: p_nom_opt, dtype: float64


# Task A

In [ ]:
setup = {'DK_1': 
                            {'OCGT': True,
                            'CCGT': True,
                            'battery storage': True,
                            'onshore wind': True,
                            'solar': True},
        'DE':  {'OCGT': True,
                'CCGT': True,
                'battery storage': True,
                'onshore wind': True,
                'solar': True} }

In [ ]:
import numpy as np
tmp.data_dict['DK']['Demand'].values.flatten()

In [ ]:
technology_data['onshorewind']['Capital_cost']



Det er noteret at:

In [ ]:
plt.plot(DataGeneration(year = 2019, region = 'DK_1').data['wind_offshore_generation_actual'].values)
plt.show()


In [ ]:
from DurationCurve import DurationCurve

# region = 'DK_1'
# region = 'DK_2'
# region = 'NO'
region = 'DE'

years = [2017, 2018, 2019, 2020]

DurationCurve(years = years, region = region)
plt.show()